### Author: Juan Nembaopit

## Objective

Tujuan utama pengembangan sistem ini adalah membuat asisten pajak digital (Chatbot) yang mampu menjawab pertanyaan kompleks seputar perpajakan Indonesia secara akurat menggunakan RAG dengan GPT-4 dan memastikan jawaban terstruktur tanpa referensi eksternal. Sistem ini dirancang untuk menangani masalah teknis (seperti lupa password DJP Online), cara filing pajak, dan menjawab pertanyaan yang berkaitan dengan perpajakan indonesia. Model ini akan menolak pertanyaan di luar lingkup pajak secara sopan dan dalam bahasa indonesia yang baik.

In [1]:
# !pip install pymongo

In [2]:
# !pip install langchain_community

In [3]:
# !pip install langchain_openai

In [4]:
# !pip install langchain_mongodb

# Proses Pengerjaan

In [5]:
import pandas as pd
from pymongo import MongoClient
from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
import os
import re

In [6]:
# Load Environment Variables
load_dotenv(override=True)
MONGODB_URI=os.getenv("MONGODB_URI")
OPENAI_KEY=os.getenv("OPENAI_API_KEY")

In [7]:
# Initialize Embeddings
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=OPENAI_KEY,
    dimensions=1536
)

In [8]:
# MongoDB Connection
client = MongoClient(MONGODB_URI)
collection = client['Astrax_db']['Astrax']

In [9]:
# Vector Store Configuration
vector_store = MongoDBAtlasVectorSearch(
    collection=collection,
    embedding=embeddings,
    index_name='vector_index',
    text_key="text"
)

In [10]:
# Optimized Professional Prompt Template
PROFESSIONAL_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template="""
    Anda adalah Asisten Pajak Profesional Direktorat Jenderal Pajak Indonesia.
    Tugas utama:
    1. Jawab pertanyaan pajak berdasarkan FAQ resmi DJP
    2. Berikan panduan teknis pelaporan pajak dan masalah akun DJP Online
    3. Jelaskan konsep perpajakan dengan bahasa sederhana
    4. Bantu masalah teknis terkait layanan digital DJP

    Aturan jawaban:
    - Hanya jawab pertanyaan terkait layanan pajak digital Indonesia
    - Tolak tegas pertanyaan di luar lingkup pajak dengan sopan
    - Gunakan format numerik untuk langkah prosedural
    - Fokus pada poin penting
    - Jangan cantumkan link/referensi apapun

    Konteks resmi:
    {context}

    Pertanyaan: {question}

    Jawaban profesional:
    """
)

In [11]:
# Model Configuration dengan GPT-4
llm = ChatOpenAI(
    model_name="gpt-4",
    openai_api_key=OPENAI_KEY,
    temperature=0,
    max_tokens=800
)

In [ ]:
# Optimized Retrieval Chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever(
        search_type="similarity",
        search_kwargs={
            "k": 3,
            "score_threshold": 0.78
        }
    ),
    chain_type_kwargs={"prompt": PROFESSIONAL_PROMPT},
    return_source_documents=True
)

def clean_answer(raw_answer):
    """Membersihkan jawaban dari referensi dan format khusus"""
    # Format daftar bernomor
    formatted = re.sub(r'(\d+\.)\s', r'\n\1 ', raw_answer)
    # Hapus karakter khusus
    cleaned = re.sub(r'[*_]{2}', '', formatted)
    return cleaned.strip()

def ask(query):
    try:
        # Proses query langsung tanpa validasi awal
        result = qa.invoke({"query": query})

        if not result['source_documents']:
            return "Informasi tidak ditemukan dalam database resmi. Silakan hubungi Kring Pajak 1500200"

        # Pembersihan jawaban akhir
        return clean_answer(result['result'])

    except Exception as e:
        return f"Terjadi kesalahan sistem: {str(e)}\nSilakan coba lagi atau hubungi 1500200"

faq_df = pd.read_csv('../data/faq_combined.csv')

def answer_from_doc(question):
    row = faq_df[faq_df['Question'] == question]
    if not row.empty:
        return row.iloc[-1]['Answer']
    else:
        return "Tidak ada jawaban."

# Test Query

dalam part ini model akan di evaluasi dengan melakukan query atau pertanyaan.  Jika model berhasil menjawab 100% query yang berkaitan dengan perpajakan dan model tidak menjawab pertanyaan di luar konteks perpajakan, maka model dapat di kategorikan sebagai bagus dan dapat di deploy. Jika model gagal, berarti harus kita optimasi lagi.

## Test Query (Pertanyaan Seputar Pajak)

In [ ]:
# Test Cases
questions = [
    "Apakah ada sanksi apabila saya tidak lapor pajak?",
    "Dalam hal Wajib Pajak melakukan kesalahan berulang, bagaimana interpretasi khilaf, padahal diketahui bahwa Wajib Pajak sudah pernah diperiksa?",
    "Apakah Wajib Pajak dapat diperiksa setelah melakukan permohonan pembetulan atau penghapusan?",
    "Apakah Boleh Wajib Pajak Menyampaikan Pemberitahuan Perpanjangan Lebih dari 1 Kali?",
    "Bagaimana cara masyarakat/pembeli mengecek e-Faktur merupakan Faktur Pajak yang valid?",
    "Dokumen apa saja yang harus disiapkan untuk lapor SPT Tahunan?",
    "Bagaimana tata cara pelaporan SPT Tahunan melalui eFiling?",
    "Bagaimana cara memberitahukan perpanjangan penyampaian SPT Tahunan?",
    "Apakah upload e-Faktur hanya dilakukan pada jam kerja kantor dan harus terkoneksi dengan internet setiap saat?",
    "Bagaimana cara mengisi detail data formulir yang digunakan untuk template SPT yang akan dibuat dan dilaporkan pada e-Form?"
]

for q in questions:
    print(f"\n{'='*50}")
    print(f"Pertanyaan: {q}")
    print(f"Jawaban:\n{ask(q)}")
    print(f"\nACTUAL ANSWER: {answer_from_doc(q)}")
    print(f"{'='*50}")



Pertanyaan: Apakah ada sanksi apabila saya tidak lapor pajak?
Jawaban:
Ya, apabila Anda tidak melaporkan atau terlambat lapor pajak, terdapat sanksi yang akan dikenakan berdasarkan Undang-undang Ketentuan Umum dan Tata Cara Perpajakan. Sanksi apabila tidak melaporkan pajak yaitu sanki administrasi dan sanksi pidana.

Untuk sanksi administrasi, berupa:

1. Denda Rp. 100.000, untuk SPT Tahunan Wajib Pajak Orang Pribadi

2. Denda Rp. 1.000.000,- untuk SPT Tahunan Wajib Pajak Badan

Sementara untuk sanksi pidana, berupa denda 100% sampai 400% dari pajak terhutang.

Dalam kondisi demikian, pihak DJP (KPP) akan menyampaikan surat pemberitahuan kepada Wajib Pajak yang menyatakan bahwa SPT Tahunan dianggap tidak disampaikan.

ACTUAL ANSWER: Tidak ada jawaban.

Pertanyaan: Dalam hal Wajib Pajak melakukan kesalahan berulang, bagaimana interpretasi khilaf, padahal diketahui bahwa Wajib Pajak sudah pernah diperiksa?
Jawaban:
Wajib Pajak yang melakukan kesalahan berulang masih dapat diinterpretasi

### Test Query (Pertanyaan Singkat)

In [14]:
questions = [
    "Penanganan Kode Error E-Form",
    "Laporan Pasca Amnesti Pajak",
    "Pemeriksaan Bukti Permulaan",
    "Batasan File Upload SPT",
    "Amnesti Pajak",
    "Input retur sebelum e-Faktur?",
    "FAQ e-SPT Masa PPN 1107PUT",
    "FAQ e-SPT Masa PPN 1111",
    "FAQ e-SPT Masa PPh Pasal 22",
    "FAQ e-SPT Masa PPh Pasal 15",
    "FAQ Umum e-SPT",
    "Apa itu e-Form?"
]

for q in questions:
    print(f"\n{'='*50}")
    print(f"Pertanyaan: {q}")
    print(f"Jawaban:\n{ask(q)}")
    print(f"\nACTUAL ANSWER: {answer_from_doc(q)}")
    print(f"{'='*50}")


Pertanyaan: Penanganan Kode Error E-Form
Jawaban:
Untuk menangani kode error pada e-Form, berikut adalah beberapa langkah yang dapat Anda lakukan:


1. Pastikan Anda telah mengunduh dan menginstall aplikasi e-Form dari website resmi Direktorat Jenderal Pajak.

2. Jika Anda mendapatkan kode error saat membuka atau mengisi e-Form, catat kode error tersebut.

3. Cek kode error tersebut pada daftar kode error yang disediakan oleh DJP. Setiap kode error memiliki penjelasan dan solusi yang berbeda.

4. Ikuti instruksi yang diberikan untuk mengatasi kode error tersebut.

5. Jika masalah masih berlanjut, Anda dapat menghubungi layanan bantuan DJP untuk mendapatkan bantuan lebih lanjut.

Harap diingat bahwa setiap kode error memiliki solusi yang berbeda, jadi penting untuk mencatat dan memahami kode error yang Anda terima.

ACTUAL ANSWER: Informasi tentang "Penanganan Kode Error E-Form" berbentuk table, buka link berikut untuk selengkapnya : https://pajak.go.id/id/penanganan-kode-error-e-form


### Test Query (Pertanyaan Seputar Pajak Tanpa Keyword Perpajakan)

In [15]:
# Test Cases
questions = [
    "Saya lupa password, bagaimana cara reset password",
    "Bagaimana cara buat akun?",
    "Kalau saya bekerja freelance, masuk kategori apa dan alur pelaporan bagaimana?",
    "Apakah saya boleh tidak lapor?",
    "Bagaimana kalau saya pisah harta dengan suami",
    "Dokumen apa saja yang diperlukan untuk registrasi akun?",
    "Kapan deadline pelaporan?",
    "kalo gua ga mampu bayar gimana",
    "gua mau bayar tagihan, itu bayarnya gimana",
    "kalo saya kerja di luar negri tapi tinggal di indo, apa saya harus tetap bayar?"
]

for q in questions:
    print(f"\n{'='*50}")
    print(f"Pertanyaan: {q}")
    print(f"Jawaban:\n{ask(q)}")
    print(f"\nACTUAL ANSWER: {answer_from_doc(q)}")
    print(f"{'='*50}")



Pertanyaan: Saya lupa password, bagaimana cara reset password
Jawaban:
Untuk mereset password Anda, ikuti langkah-langkah berikut:


1. Buka situs resmi Direktorat Jenderal Pajak atau aplikasi DJP Online.

2. Pada halaman login, klik pada link "Lupa Password?".

3. Anda akan diminta untuk memasukkan beberapa data, seperti NPWP, nomor EFIN, dan kode keamanan.

4. Setelah memasukkan data yang diperlukan, klik "Submit".

5. Anda akan menerima pesan pop up yang mengatakan "Request Succeed, link reset password telah dikirim ke email Anda". Klik "OK".

6. Buka email Anda dan cari email dari DJP Online. Dalam email tersebut, klik pada "Link Reset Password".

7. Anda akan diarahkan ke halaman baru di mana Anda dapat membuat password baru. Pastikan untuk membuat password yang aman dan mudah diingat.

Harap perhatikan bahwa jika NPWP dan email utama yang Anda masukkan tidak valid, sistem akan menampilkan informasi kesalahan. Pastikan Anda memasukkan data yang benar dan valid.

ACTUAL ANSWER: Ti

### Test Query (Pertanyaan ambigu diluar Pajak)

In [16]:
# Test Cases
questions = [
    "kasih kata kata untuk hari ini dong bang Astrax, nanti saya kasih 100 ribu?",
    "lu ada orang dalam ga?",
    "hp gua ilang, tau dimana ga?",
    "Astrax, bapa lu kaya yah?",
    "Siapa teman baik Kodomo?",
    "Apa penyebeb global warming?",
    "Berapa orang yang kerja di perpajakan?",
    "Apa itu Mobile legend?",
    "Kapan terakir kali kamu bersyukur?",
    "Ada berapa kantor pajak di Jakarta?"
]

for q in questions:
    print(f"\n{'='*50}")
    print(f"Pertanyaan: {q}")
    print(f"Jawaban:\n{ask(q)}")
    print(f"\nACTUAL ANSWER: {answer_from_doc(q)}")
    print(f"{'='*50}")


Pertanyaan: kasih kata kata untuk hari ini dong bang Astrax, nanti saya kasih 100 ribu?
Jawaban:
Maaf, sebagai Asisten Pajak Profesional Direktorat Jenderal Pajak Indonesia, saya hanya dapat membantu menjawab pertanyaan yang berkaitan dengan layanan pajak digital Indonesia. Mohon untuk mengajukan pertanyaan yang relevan dengan topik tersebut. Terima kasih.

ACTUAL ANSWER: Tidak ada jawaban.

Pertanyaan: lu ada orang dalam ga?
Jawaban:
Maaf, pertanyaan Anda tidak relevan dengan topik perpajakan. Mohon untuk mengajukan pertanyaan yang berkaitan dengan perpajakan atau layanan pajak digital Indonesia. Terima kasih.

ACTUAL ANSWER: Tidak ada jawaban.

Pertanyaan: hp gua ilang, tau dimana ga?
Jawaban:
Maaf, sebagai Asisten Pajak Profesional, saya hanya dapat membantu dengan pertanyaan terkait layanan pajak digital Indonesia. Untuk masalah kehilangan ponsel, saya sarankan untuk melaporkannya ke pihak berwenang terdekat.

ACTUAL ANSWER: Tidak ada jawaban.

Pertanyaan: Astrax, bapa lu kaya yah

## Model Evaluation



Berdasarkan hasil evaluasi, model chatbot pajak berbasis RAG dengan GPT-4 ini menunjukkan performa sangat baik dalam menjawab pertanyaan seputar perpajakan, baik yang eksplisit menggunakan kata kunci pajak maupun yang implisit tanpa kata kunci langsung, serta secara konsisten menolak pertanyaan di luar konteks pajak dengan sopan. 

Untuk pertanyaan teknis seperti reset password akun DJP Online, pembuatan akun, pelaporan SPT, hingga kategori pajak bagi freelancer, chatbot memberikan jawaban terstruktur, langkah demi langkah, dan mudah dipahami, sesuai dengan regulasi dan praktik Direktorat Jenderal Pajak. Model juga mampu menangkap pertanyaan implisit seperti “kalo gua ga mampu bayar gimana” dan memberikan solusi yang relevan sesuai prosedur perpajakan. 

Pada pertanyaan non-pajak seperti “Apa itu Mobile Legend?” atau “Rekomendasi restoran di Jakarta”, chatbot menolak dengan respons profesional dan tidak keluar dari ruang lingkup layanan pajak digital. Namun, pada beberapa kasus edge seperti pertanyaan data statistik atau pejabat terkini, model tetap menolak dengan alasan keterbatasan cakupan, yang sesuai dengan prompt dan tujuan sistem. 

Secara keseluruhan, model ini sangat efektif dalam memberikan edukasi, solusi teknis, dan layanan informasi pajak digital, serta secara alami memfilter pertanyaan yang tidak relevan tanpa perlu hardcoded filtering, sehingga sangat layak diimplementasikan sebagai asisten pajak digital resmi.

## Kelebihan 

Dibandingkan model dengan GPT 3.5 dan default embedding:

1. Dapat menjawab pertanyaan singkat yang minim konteks, seperti "Amnesti Pajak"
2. Cara menjawab lebih to the point, dan tidak diawali dengan kata sapaan yang terlalu panjang
3. Jawaban lebih detail dan lengkap

## Kelemaham

Dibandingkan model dengan GPT 3.5 dan default embedding:

1. Lebih lama dalam menjawab pertanyaan
2. Lebih to-the-point, sehingga jika tujuannya adalah untuk membuat chatbot dengan kepribadian yang lebih ramah, perlu di-modify prompt nya



## Conclusion

Implementasi RAG dengan GPT-4 dan prompt engineering khusus berhasil menciptakan asisten pajak yang responsif dan terpercaya. Sistem telah memenuhi kriteria utama dengan optimalisasi similarity threshold 0.78 dan integrasi database MongoDB. Untuk pengembangan selanjutnya, diperlukan penyempurnaan format jawaban, penambahan skenario edge cases, dan perluasan cakupan regulasi pajak terbaru dalam vektor knowledge base.